This notebook will be mainly used for the capstone project

In [247]:
import pandas as pd
import numpy as np

In [248]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [249]:
conda config --set channel_priority true


Note: you may need to restart the kernel to use updated packages.


In [250]:
conda config --set channel_priority false


Note: you may need to restart the kernel to use updated packages.


<h1>Week 3:</h1> Segmenting and Clustering Neighborhoods in Toronto

In [2]:
#!conda install -c conda-forge folium=0.5.0 --yes

In [251]:
import pandas as pd
import numpy as np
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [252]:
import requests
postal_code_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [253]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(postal_code_url,'lxml')

In [254]:
My_table = soup.find('table',{'class':'wikitable sortable'})

In [255]:
df = pd.read_html(str(My_table))[0]
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [390]:
df.shape

(103, 5)

In [256]:
df.replace('Not assigned', np.nan, inplace=True)
df.dropna(subset=["Borough"], axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [143]:
#pip install pandas-profiling

In [151]:
from pandas_profiling import ProfileReport
prof = ProfileReport(df)
prof.to_file(output_file='report.html')

Summarize dataset:   0%|          | 0/16 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [144]:
#pip install pgeocode

In [257]:
import pgeocode
nomi = pgeocode.Nominatim('ca')

latitude = []
longitude = []

for index, row in df.iterrows():
    location = nomi.query_postal_code(row[0])  # row[0] represents Postal Code value
    latitude.append(location.latitude)
    longitude.append(location.longitude)
    
df['Latitude'] = latitude
df['Longitude'] = longitude


In [260]:
df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6518,-79.5076
99,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.3830
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.7804,-79.2505
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6325,-79.4939


In [271]:
neighborhoods = df

In [272]:
neighborhoods.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889


In [273]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [274]:
neighborhoods[neighborhoods.isnull()]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
98,NaN,NaN,NaN,NaN,NaN
99,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN
101,NaN,NaN,NaN,NaN,NaN


In [275]:
neighborhoods=neighborhoods.dropna(subset=['Longitude'])

neighborhoods=neighborhoods.dropna(subset=['Latitude'])

In [276]:
map_toronto = folium.Map(location=[toronto_lat, toronto_long], zoom_start=10)

In [277]:
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h1>Week 4 & Week 5</h1> The Battle of Neighborhoods (Cont'd)

10 most common vanues from Little Italy Ontario

In [278]:
import pgeocode

nomi = pgeocode.Nominatim('ca')

latitude = []
longitude = []

for index, row in df.iterrows():
    location = nomi.query_postal_code(row[0])  
    latitude.append(location.latitude)
    longitude.append(location.longitude)

df['Latitude'] = latitude
df['Longitude'] = longitude


df.loc[df['Neighbourhood'] == "Canada Post Gateway Processing Centre", ['Latitude', 'Longitude']] = [43.636966,-79.615819]

df


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6518,-79.5076
99,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.3830
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.7804,-79.2505
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6325,-79.4939


In [292]:
from geopy.geocoders import Nominatim

address = 'Little Italy, on'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
toronto_lat = location.latitude
toronto_long = location.longitude


In [293]:

map_toronto = folium.Map(location=[toronto_lat, toronto_long], zoom_start=10)
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#8ae37b',
        fill_opacity=0.7,
        parse_html=True).add_to(map_toronto)  
    
map_toronto

In [294]:
CLIENT_ID = '#############' # your Foursquare ID
CLIENT_SECRET = '###########' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OJNHNQSWTAFVWNPDSWDND2QIECRTMFQJDNFHN2X1EVKYXV2V
CLIENT_SECRET:0HURZ0Q1IAYZOKE0TVPXYBQ0E40N3EK2UUJWZITQM1GJOBAP


In [295]:
LIMIT = 100

radius = 500

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    toronto_lat, 
    toronto_long, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=OJNHNQSWTAFVWNPDSWDND2QIECRTMFQJDNFHN2X1EVKYXV2V&client_secret=0HURZ0Q1IAYZOKE0TVPXYBQ0E40N3EK2UUJWZITQM1GJOBAP&v=20180605&ll=43.6552079,-79.4148769&radius=500&limit=100'

In [389]:
results = requests.get(url).json()
#results

In [297]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [305]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Walton,Café,43.655045,-79.414363
1,Pinky's Ca Phe,Asian Restaurant,43.655530,-79.413879
2,Bitondo's Pizzeria,Pizza Place,43.654169,-79.413400
3,Birreria Volo,Beer Bar,43.655212,-79.414678
4,P.G. Clucks Fried Chicken,Fried Chicken Joint,43.655216,-79.414571


In [306]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [313]:
nearby_venues

,name,categories,lat,lng
0,The Walton,Café,43.655045,-79.414363
1,Pinky's Ca Phe,Asian Restaurant,43.655530,-79.413879
2,Bitondo's Pizzeria,Pizza Place,43.654169,-79.413400
3,Birreria Volo,Beer Bar,43.655212,-79.414678
4,P.G. Clucks Fried Chicken,Fried Chicken Joint,43.655216,-79.414571
...,...,...,...,...
64,Motoretta,Clothing Store,43.655593,-79.412023
65,Aroma Espresso Bar,Coffee Shop,43.655851,-79.411401
66,GoodLife Fitness Toronto College and Euclid,Gym,43.655457,-79.411077
67,The Little Dog,Hot Dog Joint,43.655641,-79.412571


In [314]:
toronto_venues = getNearbyVenues(names=nearby_venues['name'],
                                   latitudes=nearby_venues['lat'],
                                   longitudes=nearby_venues['lng']
                                  )

The Walton
Pinky's Ca Phe
Bitondo's Pizzeria
Birreria Volo
P.G. Clucks Fried Chicken
The Royal Cinema
Soundscapes
Nightowl Toronto
Empire Espresso
YamChops
Wild Indigo
Red Pegasus
Hot Mess
Dolce Gelato
Rudy
Marinella Simply Italian
California Sandwiches
Pet Uno
Utopia Cafe & Grill
The Big Chill
Sotto Voce
Bar Raval
Hogtown Pub & Oysters
The Comic Book Lounge + Gallery
San Francesco Foods
Mod Club Theatre
LoPan
La Carnita
Woodlot Restaurant & Bakery
Hapa Izakaya
Kalendar Restaurant & Bistro
Bar Isabel
Il Covo
Reverie At Weldon Park
Ted's Collision
Trattoria Taverniti
Menchie's
Sicilian Sidewalk Cafe
Snakes & Lattes
The Fish Store
Burrito Boyz
DaiLo
Duff's Famous Wings
Structube
Monocle
Village Juicery
Burgatory
Monarch Tavern
Perla Restaurant
Fred Hamilton Park
El Convento Rico
Rhum Corner
Mrs. Robinson
Mullins Alehouse Pub
Trattoria Giancarlo
Riviera Bakery Ltd.
Nova Era Bakery
iRepair.ca - iPhone & iPod Repair
Tondou Ramen
Cafe Diplomatico
Weldon Park
Regina Pizzeria & Trattoria
Banga

In [315]:
print(toronto_venues.shape)
toronto_venues.head()

(4484, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Walton,43.655045,-79.414363,The Walton,43.655045,-79.414363,Café
1,The Walton,43.655045,-79.414363,Pinky's Ca Phe,43.655530,-79.413879,Asian Restaurant
2,The Walton,43.655045,-79.414363,Bitondo's Pizzeria,43.654169,-79.413400,Pizza Place
3,The Walton,43.655045,-79.414363,Birreria Volo,43.655212,-79.414678,Beer Bar
4,The Walton,43.655045,-79.414363,P.G. Clucks Fried Chicken,43.655216,-79.414571,Fried Chicken Joint


In [316]:

toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")


toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 


fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,American Restaurant,Arcade,Art Gallery,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,...,Sushi Restaurant,Taco Place,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint
0,The Walton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Walton,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Walton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Walton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Walton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [317]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,American Restaurant,Arcade,Art Gallery,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,...,Sushi Restaurant,Taco Place,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint
0,Aroma Espresso Bar,0.015385,0.015385,0.015385,0.030769,0.0,0.0,0.015385,0.0,0.076923,...,0.030769,0.046154,0.015385,0.015385,0.015385,0.000000,0.0,0.000000,0.000000,0.015385
1,Bangarang,0.016667,0.000000,0.000000,0.033333,0.0,0.0,0.016667,0.0,0.066667,...,0.033333,0.033333,0.016667,0.016667,0.016667,0.016667,0.0,0.000000,0.000000,0.016667
2,Bar Isabel,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.025000,0.0,0.050000,...,0.000000,0.000000,0.025000,0.025000,0.000000,0.025000,0.0,0.025000,0.000000,0.000000
3,Bar Raval,0.030769,0.015385,0.030769,0.030769,0.0,0.0,0.030769,0.0,0.107692,...,0.015385,0.046154,0.015385,0.015385,0.015385,0.000000,0.0,0.000000,0.000000,0.015385
4,Birreria Volo,0.014493,0.000000,0.000000,0.028986,0.0,0.0,0.028986,0.0,0.072464,...,0.000000,0.014493,0.028986,0.014493,0.000000,0.014493,0.0,0.000000,0.000000,0.014493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,Weldon Park,0.013699,0.013699,0.000000,0.041096,0.0,0.0,0.013699,0.0,0.068493,...,0.027397,0.027397,0.013699,0.013699,0.013699,0.013699,0.0,0.000000,0.013699,0.013699
65,Wild Indigo,0.014286,0.000000,0.000000,0.042857,0.0,0.0,0.028571,0.0,0.028571,...,0.028571,0.014286,0.014286,0.014286,0.014286,0.014286,0.0,0.000000,0.014286,0.014286
66,Woodlot Restaurant & Bakery,0.031746,0.015873,0.031746,0.031746,0.0,0.0,0.015873,0.0,0.111111,...,0.015873,0.047619,0.015873,0.015873,0.015873,0.000000,0.0,0.000000,0.000000,0.015873
67,YamChops,0.017544,0.000000,0.000000,0.017544,0.0,0.0,0.035088,0.0,0.087719,...,0.000000,0.000000,0.017544,0.017544,0.000000,0.017544,0.0,0.017544,0.000000,0.017544


In [358]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aroma Espresso Bar----
                venue  freq
0                Café  0.09
1                 Bar  0.08
2          Taco Place  0.05
3      Sandwich Place  0.05
4  Italian Restaurant  0.05


----Bangarang----
                venue  freq
0                Café  0.08
1                 Bar  0.07
2  Italian Restaurant  0.05
3      Sandwich Place  0.05
4         Pizza Place  0.03


----Bar Isabel----
                venue  freq
0  Italian Restaurant  0.10
1           Gastropub  0.05
2         Coffee Shop  0.05
3                Café  0.05
4        Burger Joint  0.05


----Bar Raval----
                venue  freq
0                 Bar  0.11
1                Café  0.08
2  Italian Restaurant  0.06
3          Taco Place  0.05
4   Korean Restaurant  0.03


----Birreria Volo----
                venue  freq
0  Italian Restaurant  0.12
1                 Bar  0.07
2                Café  0.06
3      Sandwich Place  0.04
4          Restaurant  0.03


----Bitondo's Pizzeria----
                ven

In [373]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [374]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aroma Espresso Bar,Café,Bar,Sandwich Place,Italian Restaurant,Taco Place,Sushi Restaurant,Korean Restaurant,Pub,Asian Restaurant,Middle Eastern Restaurant
1,Bangarang,Café,Bar,Sandwich Place,Italian Restaurant,Sushi Restaurant,Pizza Place,Middle Eastern Restaurant,Korean Restaurant,Asian Restaurant,Taco Place
2,Bar Isabel,Italian Restaurant,Gastropub,Burger Joint,Café,Bar,Coffee Shop,Japanese Restaurant,Breakfast Spot,Pet Store,Nightclub
3,Bar Raval,Bar,Café,Italian Restaurant,Taco Place,American Restaurant,Pizza Place,Pub,Record Shop,Sandwich Place,Korean Restaurant
4,Birreria Volo,Italian Restaurant,Bar,Café,Sandwich Place,Bakery,Asian Restaurant,Tapas Restaurant,Burger Joint,Park,Restaurant


In [387]:

kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)


kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10] 

array([1, 1, 3, 1, 4, 2, 0, 2, 2, 4], dtype=int32)

In [388]:
toronto_merged.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Aroma Espresso Bar,Café,Bar,Sandwich Place,Italian Restaurant,Taco Place,Sushi Restaurant,Korean Restaurant,Pub,Asian Restaurant,Middle Eastern Restaurant
1,1,Bangarang,Café,Bar,Sandwich Place,Italian Restaurant,Sushi Restaurant,Pizza Place,Middle Eastern Restaurant,Korean Restaurant,Asian Restaurant,Taco Place
2,3,Bar Isabel,Italian Restaurant,Gastropub,Burger Joint,Café,Bar,Coffee Shop,Japanese Restaurant,Breakfast Spot,Pet Store,Nightclub
3,1,Bar Raval,Bar,Café,Italian Restaurant,Taco Place,American Restaurant,Pizza Place,Pub,Record Shop,Sandwich Place,Korean Restaurant
4,4,Birreria Volo,Italian Restaurant,Bar,Café,Sandwich Place,Bakery,Asian Restaurant,Tapas Restaurant,Burger Joint,Park,Restaurant
